### 기상청 날씨 데이터 파싱
* BeautifulSoup의 find(), find_all() 함수 사용
* 파싱한 데이터를 List, Dict 에 저장
* Json file로 저장

In [11]:
import requests
from bs4 import BeautifulSoup

url = "http://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp"
res = requests.get(url)
print(res.status_code)
soup = BeautifulSoup(res.text, 'html.parser')

print('find :', type(soup.find('title')))
print(soup.find('title'))
print(soup.find('title').text)
print('------------')
print('find_all :', type(soup.find_all('title')))
print(soup.find_all('title'))
for title in soup.find_all('title'):
    print(title.text)

200
find : <class 'bs4.element.Tag'>
<title>기상청 육상 중기예보</title>
기상청 육상 중기예보
------------
find_all : <class 'bs4.element.ResultSet'>
[<title>기상청 육상 중기예보</title>, <title>전국 육상 중기예보 - 2021년 10월 07일 (목)요일 06:00 발표</title>, <title>전국 육상중기예보</title>]
기상청 육상 중기예보
전국 육상 중기예보 - 2021년 10월 07일 (목)요일 06:00 발표
전국 육상중기예보


In [14]:
print('select_one ', type(soup.select_one('title')))
print(soup.select_one('title'))
print(soup.select_one('title').text)

select_one  <class 'bs4.element.Tag'>
<title>기상청 육상 중기예보</title>
기상청 육상 중기예보


In [15]:
print('select :', type(soup.select('title')))
print(soup.select('title'))
for title in soup.select('title'):
    print(title.text)

select : <class 'bs4.element.ResultSet'>
[<title>기상청 육상 중기예보</title>, <title>전국 육상 중기예보 - 2021년 10월 07일 (목)요일 06:00 발표</title>, <title>전국 육상중기예보</title>]
기상청 육상 중기예보
전국 육상 중기예보 - 2021년 10월 07일 (목)요일 06:00 발표
전국 육상중기예보


In [26]:
location_one = soup.find('location',attrs={"wl_ver":"3"})
print(type(location_one))
province = location_one.find('province').text
print(province)
city = location_one.find('city').text
print(city)



<class 'bs4.element.Tag'>
서울ㆍ인천ㆍ경기도
서울


In [24]:
location_one2 = soup.select_one("location[wl_ver='3']")
location_one2

<location wl_ver="3">
<province>서울ㆍ인천ㆍ경기도</province>
<city>서울</city>
<data>
<mode>A02</mode>
<tmef>2021-10-10 00:00</tmef>
<wf>구름많음</wf>
<tmn>20</tmn>
<tmx>25</tmx>
<reliability></reliability>
<rnst>30</rnst>
</data>
<data>
<mode>A02</mode>
<tmef>2021-10-10 12:00</tmef>
<wf>구름많고 비</wf>
<tmn>20</tmn>
<tmx>25</tmx>
<reliability></reliability>
<rnst>90</rnst>
</data>
<data>
<mode>A02</mode>
<tmef>2021-10-11 00:00</tmef>
<wf>구름많음</wf>
<tmn>14</tmn>
<tmx>22</tmx>
<reliability></reliability>
<rnst>30</rnst>
</data>
<data>
<mode>A02</mode>
<tmef>2021-10-11 12:00</tmef>
<wf>맑음</wf>
<tmn>14</tmn>
<tmx>22</tmx>
<reliability></reliability>
<rnst>10</rnst>
</data>
<data>
<mode>A02</mode>
<tmef>2021-10-12 00:00</tmef>
<wf>맑음</wf>
<tmn>14</tmn>
<tmx>24</tmx>
<reliability></reliability>
<rnst>10</rnst>
</data>
<data>
<mode>A02</mode>
<tmef>2021-10-12 12:00</tmef>
<wf>맑음</wf>
<tmn>14</tmn>
<tmx>24</tmx>
<reliability></reliability>
<rnst>0</rnst>
</data>
<data>
<mode>A02</mode>
<tmef>2021-10-13 00:00</